## Bibliotecas

In [1]:
import numpy as np
import pandas as pd

## Leitura dos dados

In [2]:
avaliacoes = pd.read_csv("ratings.csv")
avaliacoes.head()

,UserId:ItemId,Rating
0,4baf0ac888:8b05db84f2,3
1,f3364fd7e6:8b05db84f2,2
2,72a8982204:8b05db84f2,5
3,d0171f7a9d:e40cec3cbe,5
4,e6dad08499:e40cec3cbe,5


In [3]:
novas_colunas = avaliacoes["UserId:ItemId"].str.split(":", n = 1, expand = True)

avaliacoes["UserId"]= novas_colunas[0] 
avaliacoes["ItemId"]= novas_colunas[1] 
avaliacoes.drop(columns =["UserId:ItemId"], inplace = True) 

In [4]:
avaliacoes.head()

,Rating,UserId,ItemId
0,3,4baf0ac888,8b05db84f2
1,2,f3364fd7e6,8b05db84f2
2,5,72a8982204,8b05db84f2
3,5,d0171f7a9d,e40cec3cbe
4,5,e6dad08499,e40cec3cbe


## Pré-processamento

In [5]:
media_usuarios = avaliacoes.groupby('UserId')['Rating'].mean()

In [6]:
media_usuarios_coluna = avaliacoes.groupby('UserId')['Rating'].transform('mean')

In [7]:
avaliacoes['Rating - Mean'] = avaliacoes['Rating'] - media_usuarios_coluna

In [8]:
avaliacoes.head()

,Rating,UserId,ItemId,Rating - Mean
0,3,4baf0ac888,8b05db84f2,-0.75
1,2,f3364fd7e6,8b05db84f2,0.50
2,5,72a8982204,8b05db84f2,0.00
3,5,d0171f7a9d,e40cec3cbe,0.00
4,5,e6dad08499,e40cec3cbe,1.00


In [9]:
usuarios = set(avaliacoes.UserId.values)
itens = set(avaliacoes.ItemId.values)

In [10]:
itens_para_usuarios = avaliacoes.groupby('ItemId')['UserId'].apply(list).to_dict()

In [11]:
itens_para_usuarios

{'0000c77f48': ['dfff0a0c55'],
 '000215e0ca': ['88ee37fd13'],
 '00022a9224': ['64a8b9b627', '26f6d5fe07'],
 '0003d519bf': ['b6d7d7c07d'],
 '0004b4d629': ['b2b152ce1d', '9c0dbe63c9'],
 '0004ff09c4': ['a91c1950e4'],
 '000555f117': ['964c1f1488',
  'e778213fa3',
  '7056a077ab',
  '5c61f042a9',
  'fd25e08ff0'],
 '000680fcef': ['86faed80e1',
  '5fe9d926c4',
  'd07b71438b',
  'bc8a7216a4',
  'b7adef3bad',
  'dbe09545b7'],
 '0006b91ab7': ['d773eae44e', 'b71c511ca5'],
 '0006c8650f': ['da4faae268'],
 '0006d89ff2': ['4a13d025b8', 'b9293e309c'],
 '00072fd327': ['06abe06aab', 'b89c46a54a'],
 '000821dd61': ['d2b9af2581',
  'fe018ea425',
  '939cb586c6',
  'c46ed70797',
  'de1cfc687f'],
 '0008456f00': ['ac29595f42',
  'ddb18ee909',
  'd7a32ba022',
  '7c30dea301',
  '06d60bb0ee',
  'd09405577f',
  '136006f452',
  'e5f550aa5f',
  'acdff39807',
  '7544d47106',
  '5fc8f1583d',
  '17b8325801',
  '24f862297c',
  'ec7d6a9ae6',
  '7b1ae7d4b4',
  'fbdfab5074',
  '9e51b9e630'],
 '000d425edd': ['9b1c6bb0a3', '7

In [12]:
avaliacoes_usuarios = {}

for index, row in avaliacoes.iterrows():
    user_id = row['UserId']
    item_id = row['ItemId']
    rating_norm = row['Rating - Mean']
    
    if user_id not in avaliacoes_usuarios:
        avaliacoes_usuarios[user_id] = {}
    
    # Adicione o rating do item ao dicionário do usuário
    avaliacoes_usuarios[user_id][item_id] = rating_norm

In [13]:
alvos = pd.read_csv("targets.csv")
alvos.head()

,UserId:ItemId
0,33ce7ee122:34cb28c370
1,eab9e065e5:34cb28c370
2,f785763291:34cb28c370
3,5f8185d75d:34cb28c370
4,0eeef87507:1dfcdde662


In [14]:
alvos = pd.read_csv("targets.csv")
novas_colunas = alvos["UserId:ItemId"].str.split(":", n = 1, expand = True)
alvos["UserId"] = novas_colunas[0]
alvos["ItemId"]= novas_colunas[1] 
alvos.drop("UserId:ItemId", axis = 1, inplace = True)

In [15]:
arq_saida = open("user-user-fc.csv", "w")
arq_saida.write("UserId:ItemId,Rating\n")

21

## Filtragem colaborativa User-User

In [16]:
def user_user_fc():
    
    for linha in alvos.itertuples(index=False):
    
        item_alvo = linha[1]
        usuario_alvo = linha[0]
        usuarios_consumidores_item = itens_para_usuarios[linha[1]]
    
        numerador_avaliacao = 0
        denominador_avaliacao = 0
        
        for usuario in usuarios_consumidores_item:
    
            numerador = 0
            denominador1 = 0
            denominador2 = 0
            
            itens_comuns = avaliacoes_usuarios[usuario_alvo].keys() & avaliacoes_usuarios[usuario].keys()
            
            for item in itens_comuns:
                avaliacao_usuario_alvo = avaliacoes_usuarios[usuario_alvo][item]
                avaliacao_usuario_consumidor = avaliacoes_usuarios[usuario][item]
    
                numerador += avaliacao_usuario_alvo*avaliacao_usuario_consumidor
                denominador1 += avaliacao_usuario_alvo*avaliacao_usuario_alvo
                denominador2 += avaliacao_usuario_consumidor*avaliacao_usuario_consumidor
    
            denominador = np.sqrt(denominador1)*np.sqrt(denominador2)
    
            if denominador != 0:  # Verifique se o denominador não é zero
                similaridade = numerador / denominador
            else:
                similaridade = 0
    
            if abs(similaridade) > 0.85:
                numerador_avaliacao += similaridade * avaliacoes_usuarios[usuario][item_alvo]
                denominador_avaliacao += abs(similaridade)
        
        avaliacao = media_usuarios[usuario_alvo]
    
        if denominador_avaliacao != 0:
            avaliacao += numerador_avaliacao/denominador_avaliacao
        
        avaliacao = min(5, avaliacao)
        avaliacao = max(1, avaliacao)
        avaliacao = round(avaliacao)
        
        arq_saida.write(linha[1] + ":" + linha[0] + "," + str(avaliacao)+"\n")

In [17]:
user_user_fc()
arq_saida.close()